In [20]:
# Finding missing files
import sys; sys.path.insert(0,'..')
from ppanlp import *


In [7]:
# Get all XML files
all_xml_paths = [
    os.path.join(root,fn)
    for root,dirs,fns in os.walk(path_root_eccoII)
    for fn in fns
    if fn.endswith('.xml')
]
len(all_xml_paths), all_xml_paths[0]

(105372,
 '/Users/ryanheuser/data/ecco/ECCOII/ECCOII_HistoryandGeography/XML/ECCOF0001-C00022/MONOGRAPHS/1505401300_PageText.xml')

In [8]:
# Divide this by 2 to get number of texts (approx), since 2 files per text
len(all_xml_paths) // 2

52686

In [9]:
def path2id(path):
    return os.path.basename(path).split('_')[0]

In [10]:
text_ids = sorted(list({path2id(fn) for fn in all_xml_paths}))
len(text_ids), text_ids[0]

(52686, '1298600200')

In [11]:
textid2paths = {k:{} for k in text_ids}
for path in all_xml_paths:
    assert 'PageText' in path or 'DocMetadata' in path
    key='meta' if 'DocMetadata' in path else 'text'
    textid2paths[path2id(path)][key]=path

In [ ]:
import bs4

def parse_pages(paths):
    path_meta=paths.get('meta')
    path_text=paths.get('text')

    with open(path_meta) as fmeta, open(path_text) as ftxt:
        dom_meta = bs4.BeautifulSoup(fmeta.read())
        dom_text = bs4.BeautifulSoup(ftxt.read())

    pages_meta = dom_meta('page')
    attrs_meta = ['pageid','assetid','ocrlanguage','ocr','imagelink']
    ld_meta = [{ak:page.find(ak).text for ak in attrs_meta} for page in pages_meta]

    pages_text = dom_text('page')
    ld_text = [{'pageid':page.get('id'), 'text':page.find('ocrtext').text} for page in pages_text]

    # join both on page id
    odf=pd.DataFrame(ld_meta).merge(
        pd.DataFrame(ld_text), 
        on='pageid',
        how='outer'
    )
    # quant cols?
    odf['ocr'] = pd.to_numeric(odf['ocr'],errors='coerce')
    return {d['pageid']:d for d in odf.to_dict('records')}

In [2]:
# df=pd.read_json('../data/data.hathi_pages.v3.jsonl', lines=True)

In [3]:
# df['id']=[f'{x+"_"+y.replace(" ","") if y else x}' for x,y in zip(df.source_id, df.pages_orig)]

In [4]:
# df.id.value_counts()

In [5]:
df_meta = get_ppa_metadata().reset_index()
df_meta.source_id.nunique()

6608

In [7]:
# sources=set(df.source_id)
# len(sources)

In [11]:
folders = set(os.listdir(PATH_TEXT_CORPUS_TEXTS_PAGES))
missingdf = df_meta[~df_meta.source_id.isin(folders)]

In [12]:
missingdf.source.value_counts()

source
Gale          1071
HathiTrust     512
Name: count, dtype: int64

In [13]:
missingdf.drop_duplicates('source_id').source.value_counts()

source
Gale          1000
HathiTrust     511
Name: count, dtype: int64

In [16]:
missingdf

,source_id,id,record_id,title,subtitle,sort_title,author,item_type,book_journal,pub_date,pub_place,publisher,enumcron,collections,cluster,public_notes,notes,pages_orig,pages_digital,page_count,status,source,added,updated,source_url
26,CW0110723387,5362,T102913,An abridgment of L. Murray's English grammar.,"With an appendix, containing an exemplificatio...",abridgment of L. Murray's English grammar. Wit...,"Murray, Lindley, 1745-1826",Full work,,1798.0,London,"Darton and Harvey, Gracechurch-Street; C. Law,...",,Linguistic,murrayabridgment,,1798,,,122.0,Public,Gale,2021-08-05 10:11:52,2023-03-02 13:50:11,https://link.gale.com/apps/doc/CW0110723387/EC...
27,CW0111689573,5365,T102916,An abridgment of L. Murray's English grammar.,"With an appendix, containing an exemplificatio...",abridgment of L. Murray's English grammar. Wit...,"Murray, Lindley, 1745-1826",Full work,,1797.0,York,"Wilson, Spence, and Mawman: sold also by Darto...",,Linguistic,murrayabridgment,,1797,,,106.0,Public,Gale,2021-08-05 10:11:56,2023-03-02 13:50:12,https://link.gale.com/apps/doc/CW0111689573/EC...
28,CW0112087160,5657,T69086,The abridgment of the new method of learning e...,Translated from the French of Messieurs de Por...,abridgment of the new method of learning easil...,"Lancelot, Claude, 1615?-1695",Full work,,1749.0,London,J. Nourse at the Lamb opposite Katherine-Stree...,,Literary,,,,,,319.0,Public,Gale,2021-08-05 10:23:44,2021-08-05 10:23:44,https://link.gale.com/apps/doc/CW0112087160/EC...
29,CW0113146825,5366,T108203,An abstract of English grammar and rhetoric,containing the chief principles and rules of b...,abstract of English grammar and rhetoric conta...,"Turner, Daniel, 1710-1798",Full work,,1739.0,London,"R. Hett, at the Bible and Crown in the Poultry...",,Linguistic,turnerabstract,,,,,61.0,Public,Gale,2021-08-05 10:11:57,2023-03-02 13:50:23,https://link.gale.com/apps/doc/CW0113146825/EC...
44,CW0115427928,5661,T200169,The accidence,or first rudiments of English grammar. Designe...,accidence or first rudiments of English gramma...,"Devis, Ellin, d. ca. 1784",Full work,,1793.0,London,"B. Law and Son, No. 13, Ave Maria-Lane, Ludgat...",,Linguistic,devisaccidence,,7th ed 1783,,,145.0,Public,Gale,2021-08-05 10:23:50,2023-03-02 13:49:56,https://link.gale.com/apps/doc/CW0115427928/EC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739,dul1.ark:/13960/t4hm60s5r,97,010944757,York's English grammar :,revised and adapted to Southern schools,York's English grammar : revised and adapted t...,"York, Brantley, 1805-1891",Full work,,1864.0,"Raleigh, N.C.","Branson, Farrar & Co.",,Linguistic; Literary; Typographically Unique,yorkenglish,,,,,136.0,Public,HathiTrust,2018-01-23 14:37:15,2023-04-25 10:46:43,https://hdl.handle.net/2027/dul1.ark:/13960/t4...
6740,dul1.ark:/13960/t8cg0hx8d,85,010944749,"York's English grammar,",revised and adapted to Southern schools,"York's English grammar, revised and adapted to...","York, Brantley, 1805-1891",Full work,,1865.0,"Raleigh, N.C.",Branson and Farrar,,Linguistic; Literary; Typographically Unique,yorkenglish,,,,,136.0,Public,HathiTrust,2018-01-23 14:37:11,2023-04-25 10:45:34,https://hdl.handle.net/2027/dul1.ark:/13960/t8...
6742,loc.ark:/13960/t8tb1zv5g,931,009024104,"The younger American poets,",,"younger American poets,","Rittenhouse, Jessie B. (Jessie Belle), 1869-1948",Full work,,1904.0,Boston,"Little, Brown, and company",,Literary,,,,,,410.0,Public,HathiTrust,2018-01-23 14:45:13,2019-02-27 11:33:19,https://hdl.handle.net/2027/loc.ark:/13960/t8t...
6750,CW0114873189,5899,N14389,"Youth's preceptor,",containing a grammar of the English tongue; A ...,"Youth's preceptor, containing a grammar of the...","Steele, Richard, Sir, 1672-1729",Full work,,1782.0,London,M. Smith and W. Davies,,Linguistic; Literary; Typographically Unique,,,,,,321.0,Public,Gale,2021-08-05 10:34:02,2021-08-05 10:34:02,https://link.gale.com/apps/doc/CW0114873189/EC...


In [17]:
missing_ecco = missingdf.query('source=="Gale"')
missing_ecco

,source_id,id,record_id,title,subtitle,sort_title,author,item_type,book_journal,pub_date,pub_place,publisher,enumcron,collections,cluster,public_notes,notes,pages_orig,pages_digital,page_count,status,source,added,updated,source_url
26,CW0110723387,5362,T102913,An abridgment of L. Murray's English grammar.,"With an appendix, containing an exemplificatio...",abridgment of L. Murray's English grammar. Wit...,"Murray, Lindley, 1745-1826",Full work,,1798.0,London,"Darton and Harvey, Gracechurch-Street; C. Law,...",,Linguistic,murrayabridgment,,1798,,,122.0,Public,Gale,2021-08-05 10:11:52,2023-03-02 13:50:11,https://link.gale.com/apps/doc/CW0110723387/EC...
27,CW0111689573,5365,T102916,An abridgment of L. Murray's English grammar.,"With an appendix, containing an exemplificatio...",abridgment of L. Murray's English grammar. Wit...,"Murray, Lindley, 1745-1826",Full work,,1797.0,York,"Wilson, Spence, and Mawman: sold also by Darto...",,Linguistic,murrayabridgment,,1797,,,106.0,Public,Gale,2021-08-05 10:11:56,2023-03-02 13:50:12,https://link.gale.com/apps/doc/CW0111689573/EC...
28,CW0112087160,5657,T69086,The abridgment of the new method of learning e...,Translated from the French of Messieurs de Por...,abridgment of the new method of learning easil...,"Lancelot, Claude, 1615?-1695",Full work,,1749.0,London,J. Nourse at the Lamb opposite Katherine-Stree...,,Literary,,,,,,319.0,Public,Gale,2021-08-05 10:23:44,2021-08-05 10:23:44,https://link.gale.com/apps/doc/CW0112087160/EC...
29,CW0113146825,5366,T108203,An abstract of English grammar and rhetoric,containing the chief principles and rules of b...,abstract of English grammar and rhetoric conta...,"Turner, Daniel, 1710-1798",Full work,,1739.0,London,"R. Hett, at the Bible and Crown in the Poultry...",,Linguistic,turnerabstract,,,,,61.0,Public,Gale,2021-08-05 10:11:57,2023-03-02 13:50:23,https://link.gale.com/apps/doc/CW0113146825/EC...
44,CW0115427928,5661,T200169,The accidence,or first rudiments of English grammar. Designe...,accidence or first rudiments of English gramma...,"Devis, Ellin, d. ca. 1784",Full work,,1793.0,London,"B. Law and Son, No. 13, Ave Maria-Lane, Ludgat...",,Linguistic,devisaccidence,,7th ed 1783,,,145.0,Public,Gale,2021-08-05 10:23:50,2023-03-02 13:49:56,https://link.gale.com/apps/doc/CW0115427928/EC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,CW0111185737,5891,T139433,The works,of Virgil: translated into English blank verse...,works of Virgil: translated into English blank...,Virgil,Full work,,1731.0,London,"J. Brotherton, J. Hazard, W. Meadows, T. Cox, ...",2,Literary,,,,,,422.0,Public,Gale,2021-08-05 10:33:49,2021-08-05 10:33:49,https://link.gale.com/apps/doc/CW0111185737/EC...
6685,CW0115944158,5884,T92021,The works,"of Virgil translated into English prose, As ne...","works of Virgil translated into English prose,...",Virgil,Full work,,1743.0,London,"Joseph Davidson, at the Angel in the Poultry, ...",1,Literary,virgilworksprose1,,,,,491.0,Public,Gale,2021-08-05 10:33:25,2023-03-02 13:50:23,https://link.gale.com/apps/doc/CW0115944158/EC...
6687,CW0111790820,5883,N22150,The works,"of Virgil translated into English prose, ... W...","works of Virgil translated into English prose,...",Virgil,Full work,,1770.0,London,"assignment from Joseph Davidson, for J. Beecro...",1,Literary,virgilworksprose1,,,,,423.0,Public,Gale,2021-08-05 10:33:21,2023-03-02 13:50:23,https://link.gale.com/apps/doc/CW0111790820/EC...
6711,CW0114345741,5893,T145999,The wreath; composed of selections from Sappho...,"Accompanied by a prose translation, with notes...","wreath; composed of selections from Sappho, Th...",,Full work,,1799.0,London,"T. Bensley; for White, Fleet Street; Egerton, ...",,Literary,,,,,,120.0,Public,Gale,2021-08-05 10:33:52,2021-08-05 10:33:52,https://link.gale.com/apps/doc/CW0114345741/EC...


In [19]:
dfecco=get_ecco_metadata().reset_index()
overlap = set(missing_ecco.source_id) & set(dfecco.source_id)
len(overlap)

1000

,ocr,psmid,assetid,assetidetoc,dvicollectionid,bibliographicid,unit,reel,mcode,composed,productlink,fulltitle,displaytitle,currentvolume,totalvolumes,publicationplacecity,publicationplacecomposed,totalpages,collation,author_composed,author_first,author_last,author_birthdate,author_deathdate,year,edition,editionstatement
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
CW0118040112,91.38,139800600,118040112,418040112,ECCOC0001,T6587,40,1398,0LRF,MDCCXXXV. [1735],http://gdc.galegroup.com/gdc/artemis/Monograph...,"The veneration of saints and images, as taught...","The veneration of saints and images, as taught...",0,0,London,London,55,"58,[2]p. ; 8°","Hughes, Obadiah",Obadiah,Hughes,1695,1751,NaN,NaN,NaN
CW0123835250,93.21,348003500,123835250,423835250,ECCOC0001,N16786,100,3480,0LRJ,[1768],http://gdc.galegroup.com/gdc/artemis/Monograph...,"Information for Robert Geddie junior, merchant...","Information for Robert Geddie junior, merchant...",0,0,[Edinburgh?],[Edinburgh?],20,20p. ; 4°,"Geddie, Robert, junior",Robert,Geddie,NaN,NaN,NaN,NaN,NaN
CW0111282408,91.14,282100500,111282408,411282408,ECCOC0001,T72276,81,2821,0LRK,1720,http://gdc.galegroup.com/gdc/artemis/Monograph...,Serious reflections during the life and surpri...,Serious reflections during the life and surpri...,0,0,London,London,374,"[16],270;64,63-84,[2]p.,plate : map ; 8°","Defoe, Daniel",Daniel,Defoe,1661?,1731,1720.0,NaN,NaN
CW0104382868,89.82,127001304,104382868,404382868,ECCOC0001,N7127,37,1270,0LRI,M.DCC.LIII. 1752- [1753],http://gdc.galegroup.com/gdc/artemis/Monograph...,"The independent Whig: or, a defence of primiti...","The independent Whig: or, a defence of primiti...",Volume 4,4,London,London,347,4v. ; 12°.,NaN,NaN,NaN,NaN,NaN,NaN,8,"The eighth edition, with additions and amendme..."
CW0104332779,89.31,1131300103,104332779,404332779,ECCOC0001,N8931,324,11313,0LRH,MDCCXLVIII. [1748],http://gdc.galegroup.com/gdc/artemis/Monograph...,The history of the life and times of Cardinal ...,The history of the life and times of Cardinal ...,Volume 3,4,London,London,478,"4v.,plates ; 8°","Grove, Joseph",Joseph,Grove,NaN,1764,NaN,2,"The second edition, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CB0132979036,57.64,1783800115,132979036,432979036,ECCOC0001,T105200,509,17838,2NEI,"M,DCC,XCVI. [1796]",http://gdc.galegroup.com/gdc/artemis/Monograph...,The journals of the House of Commons of the ki...,The journals of the House of Commons of the ki...,Volume 15,19,Dublin,Dublin,1032,19v. ; 2°,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CB0132980068,57.74,1783800116,132980068,432980068,ECCOC0001,T105200,509,17838,2NEI,"M,DCC,XCVI. [1796]",http://gdc.galegroup.com/gdc/artemis/Monograph...,The journals of the House of Commons of the ki...,The journals of the House of Commons of the ki...,Volume 16,19,Dublin,Dublin,995,19v. ; 2°,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CB0132981063,55.11,1783800117,132981063,432981063,ECCOC0001,T105200,509,17838,2NEI,"M,DCC,XCVI. [1796]",http://gdc.galegroup.com/gdc/artemis/Monograph...,The journals of the House of Commons of the ki...,The journals of the House of Commons of the ki...,Volume 17,19,Dublin,Dublin,1538,19v. ; 2°,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
